In [ ]:
import os
import numpy as np
from tqdm.notebook import tqdm, trange

import torch
from haloflow import data as D
from haloflow import util as U

In [ ]:
import corner as DFM
# --- plotting ---
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

In [ ]:
if torch.cuda.is_available(): device = 'cuda'
else: device = 'cpu'

## read $q_\phi(Y\,|\,X)$

In [ ]:
qphis = U.read_best_ndes('h2.v1.TNG50.mags_morph', n_ensemble=5, device=device, 
                         dat_dir='/scratch/gpfs/chhahn/haloflow/hf2/npe', verbose=True)
Y_test, X_test = D.hf2_centrals('test', 'mags_morph', sim='TNG50')

## some examples

In [ ]:
n_sample = 10000
for i in np.random.choice(np.arange(X_test.shape[0]), size=5, replace=False): 
    y_samp = []
    for qphi in qphis: 
        _samp = qphi.sample((int(n_sample/len(qphis)),),
                               x=torch.tensor(X_test[i], dtype=torch.float32).to(device), 
                               show_progress_bars=True)
        y_samp.append(_samp.detach().cpu().numpy())
    y_nde = np.concatenate(np.array(y_samp), axis=0)
    
    
    fig = DFM.corner(y_nde, color='C0', plot_datapoints=False, plot_density=False)
    DFM.overplot_points(fig, [Y_test[i]], color='r')
    DFM.overplot_lines(fig, Y_test[i], color='r')

## coverage test

In [ ]:
n_sample = 1000

ranks = []
y_nde = [] 
for i in trange(X_test.shape[0]): 
    y_samp = []
    for qphi in qphis: 
        _samp = qphi.sample((int(n_sample/len(qphis)),),
                               x=torch.tensor(X_test[i], dtype=torch.float32).to(device), 
                               show_progress_bars=False)
        y_samp.append(_samp.detach().cpu().numpy())
    y_nde.append(np.concatenate(np.array(y_samp), axis=0)) 
    
    _ranks = []
    for i_dim in range(y_nde[-1].shape[1]): 
        _ranks.append(np.mean(y_nde[-1][:,i_dim].flatten() < Y_test[i, i_dim]))
    ranks.append(_ranks)
    
ranks = np.array(ranks)
y_nde = np.array(y_nde)

In [ ]:
fig = plt.figure(figsize=(12,3.5))
sub = fig.add_subplot(121)

_ = sub.hist(ranks[:,0], range=(0., 1), bins=20, histtype='step', density=True, linewidth=2)

sub.plot([0., 1.], [1., 1.], c='k', ls='--')
sub.text(0.05, 0.95, r'$\log M_*$', fontsize=20, transform=sub.transAxes, ha='left', va='top')
sub.set_xlabel('rank statistics', fontsize=20)
sub.set_xlim(0., 1.)
sub.set_ylim(0., 3.)
sub.set_yticks([])

sub = fig.add_subplot(122)

_ = sub.hist(ranks[:,1], range=(0., 1), bins=20, histtype='step', density=True, linewidth=2)

sub.plot([0., 1.], [1., 1.], c='k', ls='--')
sub.text(0.05, 0.95, r'$\log M_h$', fontsize=20, transform=sub.transAxes, ha='left', va='top')
sub.set_xlabel('rank statistics', fontsize=20)
sub.set_xlim(0., 1.)
sub.set_ylim(0., 3.)
sub.set_yticks([])

In [ ]:
from tarp import get_drp_coverage

In [ ]:
alpha, ecp = get_drp_coverage(np.swapaxes(y_nde, 0, 1), Y_test, references="random", metric="euclidean")


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
ax.plot([0, 1], [0, 1], ls="--", color="k")
ax.plot(alpha, ecp, c='C0')
ax.legend(loc='lower right', fontsize=15)
ax.set_ylabel("Expected Coverage", fontsize=20)
ax.set_ylim(0., 1.)
ax.set_xlabel("Credibility Level", fontsize=20)
ax.set_xlim(0., 1.)